In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asus\\Desktop\\Deep Learning project\\Chest-Cancer-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\asus\\Desktop\\Deep Learning project\\Chest-Cancer-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        # Using tf.keras.utils.image_dataset_from_directory (modern API)
        image_size = tuple(self.config.params_image_size[:-1])
        batch_size = self.config.params_batch_size
        
        # Validation dataset
        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.20,
            subset="validation",
            seed=123,
            image_size=image_size,
            batch_size=batch_size,
            shuffle=False
        )
        
        # Normalize validation data
        normalization_layer = tf.keras.layers.Rescaling(1./255)
        self.valid_generator = self.valid_generator.map(lambda x, y: (normalization_layer(x), y))
        
        # Training dataset
        self.train_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.20,
            subset="training",
            seed=123,
            image_size=image_size,
            batch_size=batch_size,
            shuffle=True
        )
        
        # Normalize training data
        self.train_generator = self.train_generator.map(lambda x, y: (normalization_layer(x), y))
        
        # Apply data augmentation if enabled
        if self.config.params_is_augmentation:
            data_augmentation = tf.keras.Sequential([
                tf.keras.layers.RandomFlip("horizontal"),
                tf.keras.layers.RandomRotation(0.2),
                tf.keras.layers.RandomZoom(0.2),
                tf.keras.layers.RandomTranslation(0.2, 0.2),
            ])
            self.train_generator = self.train_generator.map(
                lambda x, y: (data_augmentation(x, training=True), y)
            )
        
        # Optimize performance
        AUTOTUNE = tf.data.AUTOTUNE
        self.train_generator = self.train_generator.prefetch(buffer_size=AUTOTUNE)
        self.valid_generator = self.valid_generator.prefetch(buffer_size=AUTOTUNE)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        # Calculate class weights to handle imbalanced dataset
        # This gives more importance to minority class during training
        # For 2:1 ratio (300 adenocarcinoma : 150 normal), weights will be auto-calculated
        # Example: if class 0 has 150 samples and class 1 has 300 samples,
        # class_weight = {0: 2.0, 1: 1.0} (minority class gets higher weight)
        from sklearn.utils import class_weight
        import numpy as np
        
        # Get class labels from training data
        class_labels = np.concatenate([y for x, y in self.train_generator], axis=0)
        class_labels = np.argmax(class_labels, axis=1)  # Convert one-hot to labels
        
        # Calculate balanced class weights
        class_weights = class_weight.compute_class_weight(
            class_weight='balanced',
            classes=np.unique(class_labels),
            y=class_labels
        )
        class_weight_dict = dict(enumerate(class_weights))
        
        print(f"Class weights applied: {class_weight_dict}")
        print("This helps balance the 2:1 dataset ratio (300 adenocarcinoma : 150 normal)")
        
        # Train with class weights for balanced learning
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator,
            class_weight=class_weight_dict
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [ ]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e